In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import xarray as xr
import os
import torch
from functools import reduce 

torch.device("cuda" if torch.cuda.is_available() else "cpu")

LOW_RES_SAMPLE_PATH = "data/ClimSim_low-res/train/"
LOW_RES_GRID_PATH = "data/ClimSim_low-res/ClimSim_low-res_grid-info.nc"
ZARR_PATH = "data/ClimSim_low-res.zarr"

In [ ]:
class ClimSimMLP(nn.Module):
    def __init__(self, input_dim=556, output_tendancies_dim=120, output_surface_dim=8):
        super(ClimSimMLP, self).__init__()
        
        # Hidden Layers: [768, 640, 512, 640, 640]
        self.layer1 = nn.Linear(input_dim, 768)
        self.layer2 = nn.Linear(768, 640)
        self.layer3 = nn.Linear(640, 512)
        self.layer4 = nn.Linear(512, 640)
        self.layer5 = nn.Linear(640, 640)
        

        self.last_hidden = nn.Linear(640, 128)
        
        # --- Output Heads ---
        self.head_tendencies = nn.Linear(128, output_tendancies_dim)
        self.head_surface = nn.Linear(128, output_surface_dim)
        
        # LeakyReLU alpha=0.15
        self.activation = nn.LeakyReLU(0.15)

    def forward(self, x):
        # Pass through the 5 main hidden layers
        x = self.activation(self.layer1(x))
        x = self.activation(self.layer2(x))
        x = self.activation(self.layer3(x))
        x = self.activation(self.layer4(x))
        x = self.activation(self.layer5(x))
        
        # Pass through the fixed 128 layer
        x = self.activation(self.last_hidden(x))
        
        # Output 1: Tendencies (Linear activation)
        out_linear = self.head_tendencies(x)
        
        # Output 2: Surface variables (ReLU activation)
        out_relu = F.relu(self.head_surface(x))
        
        # Concatenate along the feature dimension (dim=1)
        return torch.cat([out_linear, out_relu], dim=1)

        return out_linear

@torch.no_grad()
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_samples = 0

    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        batch_size = inputs.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size

    average_loss = total_loss / total_samples
    return average_loss

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    total_samples = 0
    
    pbar = tqdm(dataloader, desc="Training", unit="batch")

    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        batch_size = inputs.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size
        
        # Update progress bar description with current loss
        pbar.set_postfix({"loss": f"{loss.item():.6f}"})

    return total_loss / total_samples

In [30]:
import torch
import xarray as xr
import numpy as np
from torch.utils.data import Dataset

class ClimSimZarrDataset(Dataset):
    def __init__(self, zarr_path, grid_path, features, transform=None):
        self.zarr_path = zarr_path

        self.features = features
        self.features_list = self.__get_features__()

        print(self.features_list)

        self.ds = xr.open_zarr(zarr_path)[self.features_list]
        self.grid = xr.open_dataset(grid_path)
        self.transform = transform
        
        self.length = self.ds.dims['sample']
        
        self.input_vars = [v for v in self.ds.data_vars if 'in' in v]
        self.output_vars = [v for v in self.ds.data_vars if 'out' in v]

    def __len__(self):
        return self.length
    
    def __get_features__(self):
        feat = np.concat([self.features["features"]["tendancies"], self.features["features"]["surface"]])
        target = np.concat([self.features["target"]["tendancies"], self.features["target"]["surface"]])
        return np.concat([feat, target])

    def __getitem__(self, idx):
        sample = self.ds.isel(sample=idx)

        x = sample[self.input_vars].to_array().values.astype(np.float32)
        y = sample[self.output_vars].to_array().values.astype(np.float32)
        
        if self.transform:
            x = self.transform(x)
            
        return torch.from_numpy(x), torch.from_numpy(y)
    
    def get_models_dims(self, variables_dict):
        features_tend = variables_dict["features"]["tendancies"]
        features_surf = variables_dict["features"]["surface"]
        
        target_tend = variables_dict["target"]["tendancies"]
        target_surf = variables_dict["target"]["surface"]

        in_tend_dim = sum([self.ds[var].shape[-1] if len(self.ds[var].shape) > 1 else 1 for var in features_tend])
        in_surf_dim = len(features_surf) # Les variables de surface sont des scalaires (1 dim par var)
        
        out_tend_dim = sum([self.ds[var].shape[-1] if len(self.ds[var].shape) > 1 else 1 for var in target_tend])
        out_surf_dim = len(target_surf)

        return {
            "input_total": in_tend_dim + in_surf_dim,
            "output_tendancies": out_tend_dim,
            "output_surface": out_surf_dim
        }

In [34]:
ds = xr.open_zarr(ZARR_PATH)
print(ds.data_vars)

Data variables:
    in_cam_in_ALDIF      (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_ALDIR      (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_ASDIF      (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_ASDIR      (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_ICEFRAC    (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_LANDFRAC   (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_LWUP       (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_OCNFRAC    (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_SNOWHICE   (sample, ncol) float64 33MB dask.array<chunksize=(50, 384), meta=np.ndarray>
    in_cam_in_SNOWHLAND  (sample, ncol) float64 33MB dask.array<ch

In [ ]:
BATCH_SIZE = 1000
N_EPOCHS = 10

FEATURES = {
    "features" :{
        "tendancies" : ["in_state_t", "in_state_q0001", "in_state_u", "in_state_v"],
        "surface" : ["in_pbuf_COSZRS", "in_pbuf_LHFLX", "in_pbuf_SHFLX", "in_pbuf_TAUX", "in_pbuf_TAUY", "in_pbuf_SOLIN"],
    },  
    "target" :{
        "tendancies" : ["out_state_t"],
        "surface" : ["out_cam_out_SOLL"]
    }
}

dataset = ClimSimZarrDataset(ZARR_PATH, LOW_RES_GRID_PATH, FEATURES)

model_dims = ClimSimZarrDataset.get_models_dims(variables)

model = ClimSimMLP(input_dim=model_dims["input_total"], output_tendancies_dim=model_dims["output_tendancies"], output_surface_dim=model_dims["output_surface"])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

['in_state_t' 'in_state_q0001' 'in_state_u' 'in_state_v' 'in_pbuf_COSZRS'
 'in_pbuf_LHFLX' 'in_pbuf_SHFLX' 'in_pbuf_TAUX' 'in_pbuf_TAUY'
 'in_pbuf_SOLIN' 'out_state_t' 'out_pbuf_LHFLX' 'out_pbuf_SHFLX'
 'out_pbuf_TAUX' 'out_pbuf_TAUY' 'out_pbuf_SOLIN']


KeyError: np.str_('out_pbuf_LHFLX')

In [20]:
train, test = train_test_split(dataset,  test_size=0.2, random_state=42)

train_loader = torch.utils.data.DataLoader(
    train, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=4,
)

test_loader = torch.utils.data.DataLoader(
    test, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=4,
)

KeyboardInterrupt: 

In [23]:
import time
import torch
import numpy as np

# 1. Vérifie que ta classe utilise 'sample=idx' et non 'time=idx'
# (Voir notre correction précédente)

# 2. Création du DataLoader de test (on prend le train_loader avec shuffle=True)
start_time = time.time()

# On récupère le premier batch
try:
    # iter() initialise le chargement, next() récupère le premier groupe
    iterator = iter(train_loader)
    batch_x, batch_y = next(iterator)
    
    end_time = time.time()
    batch_duration = end_time - start_time

    print(f"--- Rapport de Performance ---")
    print(f"Temps pour charger le 1er batch : {batch_duration:.2f} secondes")
    print(f"Taille du batch (X) : {batch_x.shape}")
    print(f"Taille du batch (Y) : {batch_y.shape}")

    # 3. Estimation pour une époque complète
    num_batches = len(train_loader)
    total_est_min = (batch_duration * num_batches) / 60
    print(f"Nombre de batchs total : {num_batches}")
    print(f"Temps estimé pour UNE époque : {total_est_min:.1f} minutes")
    
    if batch_duration > 1.0:
        print("\n⚠️ ALERTE : Le chargement est lent. Ton GPU va s'ennuyer.")
        print("Conseil : Charge tes variables en RAM (NumPy) avant le Split.")
    else:
        print("\n✅ Vitesse correcte. Tu peux lancer l'entraînement.")

except Exception as e:
    print(f"Erreur lors du test : {e}")

Erreur lors du test : name 'train_loader' is not defined


In [99]:
for epoch in range(N_EPOCHS):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device="cpu")
    val_loss = evaluate_model(model, test_loader, criterion, device="cpu")
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Training:   0%|          | 0/107 [00:00<?, ?batch/s]/home/alexandre-tonon/anaconda3/envs/pie_env/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([2880, 0])) that is different to the input size (torch.Size([2880, 60])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Training:   0%|          | 0/107 [00:00<?, ?batch/s]


RuntimeError: The size of tensor a (60) must match the size of tensor b (0) at non-singleton dimension 1

In [1]:
import numpy as np

df = np.load("ClimSimLowResShards/X_shard_0.npy")

In [2]:
import xarray as xr
xr.open_dataset("data/ClimSim_low-res.zarr")

<xarray.Dataset> Size: 33GB
Dimensions:              (sample: 10872, ncol: 384, lev: 60)
Dimensions without coordinates: sample, ncol, lev
Data variables: (12/45)
    in_cam_in_ALDIF      (sample, ncol) float64 33MB ...
    in_cam_in_ALDIR      (sample, ncol) float64 33MB ...
    in_cam_in_ASDIF      (sample, ncol) float64 33MB ...
    in_cam_in_ASDIR      (sample, ncol) float64 33MB ...
    in_cam_in_ICEFRAC    (sample, ncol) float64 33MB ...
    in_cam_in_LANDFRAC   (sample, ncol) float64 33MB ...
    ...                   ...
    out_state_q0003      (sample, lev, ncol) float64 2GB ...
    out_state_t          (sample, lev, ncol) float64 2GB ...
    out_state_u          (sample, lev, ncol) float64 2GB ...
    out_state_v          (sample, lev, ncol) float64 2GB ...
    out_tod              (sample) int32 43kB ...
    out_ymd              (sample) int32 43kB ...
Attributes:
    ne:        4
    fv_nphys:  2
    calendar:  NO_LEAP